<a href="https://colab.research.google.com/github/z1qsx/myANLP/blob/master/NLP_week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### 1.File reading 

In [0]:
import os,random,math
from nltk import word_tokenize as tokenize
import operator
import nltk
nltk.download('punkt')

TRAINING_DIR="/content/gdrive/My Drive/csv/nlp/sentence-completion/Holmes_Training_Data"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2.Spliting training files and heldout files

In [0]:
def get_training_testing(training_dir=TRAINING_DIR,split=0.5):
    filenames=os.listdir(training_dir)
    n=len(filenames) 
    print("There are {} files in the training directory: {}".format(n,training_dir))
    #random.seed(53) #if you want the same random split every time 
    random.shuffle(filenames) 
    index=int(n*split) 
    return(filenames[:index],filenames[index:])
trainingfiles,heldoutfiles=get_training_testing()

There are 522 files in the training directory: /content/gdrive/My Drive/csv/nlp/sentence-completion/Holmes_Training_Data


### 3. Building the language model

In [0]:
class language_model(object):
    
    def __init__(self,trainingdir=TRAINING_DIR,files=[]):
        self.training_dir=trainingdir
        self.files=files
        self.train()
        
    def train(self):    
        self.unigram={}
        self.bigram={}

        self._processfiles()
        self._make_unknowns()
        self._discount()
        self._convert_to_probs()
        
        
    
    def _processline(self,line):
        tokens=["__START"]+tokenize(line)+["__END"]
        previous="__END"
        
        for token in tokens:
            self.unigram[token]=self.unigram.get(token,0)+1
            
            current=self.bigram.get(previous,{})
            
            current[token]=current.get(token,0)+1

            self.bigram[previous]=current
            
            previous=token
            
    
    def _processfiles(self):
        for afile in self.files:
            print("Processing {}".format(afile))
            try:
                with open(os.path.join(self.training_dir,afile)) as instream:
                    for line in instream:
                        line=line.rstrip()
                        if len(line)>0:
                            self._processline(line)
            except UnicodeDecodeError:
                print("UnicodeDecodeError processing {}: ignoring rest of file".format(afile))
      
            
    def _convert_to_probs(self):
        
        self.unigram={k:v/sum(self.unigram.values()) for (k,v) in self.unigram.items()}
        self.bigram={key:{k:v/sum(adict.values()) for (k,v) in adict.items()} for (key,adict) in self.bigram.items()}
        
    
    
    def _make_unknowns(self,known=2):
      unknown=0
      for (k,v) in list(self.unigram.items()):
          if v<known:
              del self.unigram[k]
              self.unigram["__UNK"]=self.unigram.get("__UNK",0)+v
      for (k,adict) in list(self.bigram.items()):
          for (kk,v) in list(adict.items()):
              isknown=self.unigram.get(kk,0)
              if isknown==0:
                  adict["__UNK"]=adict.get("__UNK",0)+v
                  del adict[kk]
          isknown=self.unigram.get(k,0)
          if isknown==0:
              del self.bigram[k]
              current=self.bigram.get("__UNK",{})
              current.update(adict)
              self.bigram["__UNK"]=current

          else:
              self.bigram[k]=adict
    
    
    def _discount(self,discount=0.75):
        #discount each bigram count by a small fixed amount
        self.bigram={k:{kk:value-discount for (kk,value) in adict.items()}for (k,adict) in self.bigram.items()}

        #for each word, store the total amount of the discount so that the total is the same 
        #i.e., so we are reserving this as probability mass
        for k in self.bigram.keys():
            lamb=len(self.bigram[k])
            self.bigram[k]["__DISCOUNT"]=lamb*discount

        #work out kneser-ney unigram probabilities
        #count the number of contexts each word has been seen in
        self.kn={}
        for (k,adict) in self.bigram.items():
            for kk in adict.keys():
                self.kn[kk]=self.kn.get(kk,0)+1
    

In [0]:
MAX_FILES=1
my_relsuts=language_model(files=trainingfiles[:MAX_FILES])

Processing HYDEA10.TXT


In [0]:
class language_model_get_prob(language_model):
  
  def get_prob(self,token,context="",methodparams={}):
    
    if methodparams.get("method","unigram")=="unigram":
        return self.unigram.get(token,self.unigram.get("__UNK",0))
    else:
        if methodparams.get("smoothing","kneser-ney")=="kneser-ney":
            unidist=self.kn
        else:
            unidist=self.unigram
            
        bigram=self.bigram.get(context[-1],self.bigram.get("__UNK",{}))
        big_p=bigram.get(token,bigram.get("__UNK",0))
        lmbda=bigram["__DISCOUNT"]
        uni_p=unidist.get(token,unidist.get("__UNK",0))
        #print(big_p,lmbda,uni_p)
        p=big_p+lmbda*uni_p            
        return p

  
    

In [0]:
prob=language_model_get_prob(files=trainingfiles[:MAX_FILES])

Processing HYDEA10.TXT


In [0]:
class Person(object):
  def __init__(self,name,sex):
    self.name = name
    self.sex = sex

  def print_title(self):
    if self.sex == "male":
      print("man")
    elif self.sex == "female":
      print("woman")
            
class Child(Person):
  pass

class Baby(Child):
  pass

In [0]:
s=Child

### 4. Generating the sentence


In [0]:
class language_model_sentence(language_model_get_prob):

  def nextlikely(self,k=1,current="",method="unigram"):
      #use probabilities according to method to generate a likely next sequence
      #choose random token from k best
      blacklist=["__START","__UNK","__DISCOUNT"]

      if method=="unigram":
          dist=self.unigram
      else:
          dist=self.bigram.get(current,self.bigram.get("__UNK",{}))

      #sort the tokens by unigram probability
      mostlikely=sorted(list(dist.items()),key=operator.itemgetter(1),reverse=True)
      
      #filter out any undesirable tokens
      filtered=[w for (w,p) in mostlikely if w not in blacklist]
      #choose one randomly from the top k
      res=random.choice(filtered[:k])
      return res

  def generate(self,k=1,end="__END",limit=20,method="bigram",methodparams={}):
      if method=="":
          method=methodparams.get("method","bigram")
      current="__START"
      tokens=[]
      while current!=end and len(tokens)<limit:
          current=self.nextlikely(k=k,current=current,method=method)
          tokens.append(current)
      return " ".join(tokens[:-1])

### Sorted method

In [0]:
a = [('john', 'A', 15), ('jane', 'B', 12), ('dave', 'C', 10)]

sorted(a, key=operator.itemgetter(1),reverse=True)

[('dave', 'C', 10), ('jane', 'B', 12), ('john', 'A', 15)]

In [0]:
sents=language_model_sentence(files=trainingfiles[:MAX_FILES])

Processing HYDEA10.TXT


5. 

In [0]:
class language_model_perplexity(language_model_get_prob):
  
  def _convert_to_probs(self):
        
    self.unigram={k:v/sum(self.unigram.values()) for (k,v) in self.unigram.items()}
    self.bigram={key:{k:v/sum(adict.values()) for (k,v) in adict.items()} for (key,adict) in self.bigram.items()}
    self.kn={k:v/sum(self.kn.values()) for (k,v) in self.kn.items()}

  def compute_prob_line(self,line,methodparams={}):
    #this will add _start to the beginning of a line of text
    #compute the probability of the line according to the desired model
    #and returns probability together with number of tokens

    tokens=["__START"]+tokenize(line)+["__END"]
    acc=0
    for i,token in enumerate(tokens[1:]):
        acc+=math.log(self.get_prob(token,tokens[:i+1],methodparams))
    return acc,len(tokens[1:])

  def compute_probability(self,filenames=[],methodparams={}):
      #computes the probability (and length) of a corpus contained in filenames
      if filenames==[]:
          filenames=self.files

      total_p=0
      total_N=0
      for i,afile in enumerate(filenames):
          print("Processing file {}:{}".format(i,afile))
          try:
              with open(os.path.join(self.training_dir,afile)) as instream:
                  for line in instream:
                      line=line.rstrip()
                      if len(line)>0:
                          p,N=self.compute_prob_line(line,methodparams=methodparams)
                          total_p+=p
                          total_N+=N
          except UnicodeDecodeError:
              print("UnicodeDecodeError processing file {}: ignoring rest of file".format(afile))
      return total_p,total_N

  def compute_perplexity(self,filenames=[],methodparams={"method":"bigram","smoothing":"kneser-ney"}):

      #compute the probability and length of the corpus
      #calculate perplexity
      #lower perplexity means that the model better explains the data

      p,N=self.compute_probability(filenames=filenames,methodparams=methodparams)
      #print(p,N)
      pp=math.exp(-p/N)
      return pp  

In [0]:
pre_unkonw=language_model_perplexity(files=trainingfiles[:MAX_FILES])

Processing HYDEA10.TXT


In [0]:
bigram=pre_unkonw.bigram

### Compute_perplexity

In [0]:
tokens=["__START"]+tokenize(line)+["__END"]
pre=0
for i,token in enumerate(tokens[1:]):
    pre+=math.log(self.get_prob(token,tokens[:i+1],methodparams))
return pre,len(tokens[1:])

In [0]:
p,N=self.compute_prob_line(line,methodparams=methodparams)
total_p+=p
total_N+=N

In [0]:
p,N=self.compute_probability(filenames=filenames,methodparams=methodparams)
#print(p,N)
pp=math.exp(-p/N)

### Discount

In [0]:
 for k in self.bigram.keys():
    lamb=len(self.bigram[k])
    self.bigram[k]["__DISCOUNT"]=lamb*discount

In [0]:
self.kn={k:v/sum(self.kn.values()) for (k,v) in self.kn.items()}